# Quick Success: Materials Search with Deep Learning

The goal of this quick success is simple, yet an actual implementation may take some time. We are going to write an Artificial Neural Network to predict the materials property. As a basic library for design the network we will use Torch which is the most convenient neural network environment when the work involves defining new layers.

The list of files in your current directory should be:

- This notebook
- quicksuccess_mining.ipynb
- quicksuccess_modules.ipynb
- quicksuccess_net.ipynb
- qucksuccess_train.ipynb

Main libraries:

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys, time
import torch
from torch.utils.data.sampler import SubsetRandomSampler
from IPython import display

In [3]:
# (re-)load modules
%run quicksuccess_mining.ipynb
%run quicksuccess_modules.ipynb
%run quicksuccess_net.ipynb
%run quicksuccess_train.ipynb

## Data mining

In [4]:
dataset = CIFData(root_dir = "superconductors",
                  max_num_nbr = 12,
                  radius=8,
                  dmin=0,
                  step=0.2)

In [5]:
orig_atom_fea_len = dataset[0][0][0].shape[-1]
nbr_fea_len = dataset[0][0][1].shape[-1]
print("Number of structures: {}".format(len(dataset)))
print("Number of features describing one atom: {}".format(orig_atom_fea_len))
print("Number of features describing neighbours: {}".format(nbr_fea_len))

Number of structures: 291
Number of features describing one atom: 92
Number of features describing neighbours: 41


In [15]:
sample_data_list = [dataset[i] for i in range(len(dataset))]
_, sample_target, _ = collate_pool(sample_data_list)
normalizer = Normalizer(sample_target)

In [6]:
indices = list(range(len(dataset)))
train_val_ratio = 0.7
train_sampler = SubsetRandomSampler(indices[:int(train_val_ratio*len(dataset))])
val_sampler = SubsetRandomSampler(indices[int(train_val_ratio*len(dataset)):])
train_loader = DataLoader(dataset, batch_size=64,
                          sampler=train_sampler,
                          collate_fn=collate_pool)
val_loader = DataLoader(dataset, batch_size=64,
                        sampler=val_sampler,
                        collate_fn=collate_pool)

## Creating model

In [16]:
model = CrystalGraphConvNet(orig_atom_fea_len, nbr_fea_len)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0)
#normalizer = Normalizer(torch.zeros(1))
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1)
model.cuda()

CrystalGraphConvNet(
  (embedding): Linear(in_features=92, out_features=64, bias=True)
  (convs): ModuleList(
    (0): ConvLayer(
      (fc_full): Linear(in_features=169, out_features=128, bias=True)
      (sigmoid): Sigmoid()
      (softplus1): Softplus(beta=1, threshold=20)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (softplus2): Softplus(beta=1, threshold=20)
    )
    (1): ConvLayer(
      (fc_full): Linear(in_features=169, out_features=128, bias=True)
      (sigmoid): Sigmoid()
      (softplus1): Softplus(beta=1, threshold=20)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (softplus2): Softplus(beta=1, threshold=20)
    )
    (2): ConvLayer(
      (fc_full): Linear(in_features=169, out_featu

In [18]:
for i, (input, target, _) in enumerate(train_loader):
    input_var = (input[0].cuda(async=True),
                 input[1].cuda(async=True),
                 input[2].cuda(async=True),
                 [crys_idx.cuda(async=True) for crys_idx in input[3]])
    target_normed = normalizer.norm(target)
    target_var = target_normed.cuda(async=True)
    print(target_normed)

tensor([[ 0.3718],
        [ 1.3097],
        [ 0.7810],
        [-0.0731],
        [-0.8381],
        [-0.7009],
        [-0.2357],
        [-1.1305],
        [ 0.7378],
        [-0.0375],
        [-0.5331],
        [-0.9347],
        [ 0.0184],
        [ 2.3366],
        [ 1.5258],
        [ 0.4709],
        [ 0.0057],
        [-0.7746],
        [ 1.6427],
        [ 1.3097],
        [ 1.0758],
        [-0.9780],
        [-0.0019],
        [-0.5484],
        [-0.5484],
        [ 0.6641],
        [ 0.1125],
        [ 0.0540],
        [-0.9957],
        [ 0.2777],
        [-0.0095],
        [-0.8864],
        [ 0.2447],
        [ 2.9466],
        [ 0.3870],
        [-0.4670],
        [ 2.9339],
        [ 2.9339],
        [-0.0731],
        [ 0.3489],
        [-0.8229],
        [ 0.0972],
        [ 0.1277],
        [ 0.1735],
        [-0.4543],
        [-1.0567],
        [-0.8763],
        [-0.3298],
        [ 0.1277],
        [-0.6068],
        [-1.1305],
        [-0.6577],
        [-1.

In [19]:
best_mae_error = 1e10
epochs = 10

for epoch in range(epochs):
        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, normalizer)

        # evaluate on validation set
        mae_error = validate(val_loader, model, criterion, normalizer)

        if mae_error != mae_error:
            print('Exit due to NaN')
            quit()

        #scheduler.step()

        # remember the best mae_eror and save checkpoint
        is_best = mae_error < best_mae_error
        best_mae_error = min(mae_error, best_mae_error)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_mae_error': best_mae_error,
            'optimizer': optimizer.state_dict(),
            'normalizer': normalizer.state_dict(),
        }, is_best)

Epoch: [0][0/4]	Time 1.086 (1.086)	Data 0.002 (0.002)	Loss 0.9072 (0.9072)	MAE 0.312 (0.312)
Epoch: [0][1/4]	Time 0.426 (0.756)	Data 0.003 (0.003)	Loss 2.4832 (1.6952)	MAE 0.450 (0.381)
Epoch: [0][2/4]	Time 0.358 (0.623)	Data 0.003 (0.003)	Loss 3.2841 (2.2248)	MAE 0.657 (0.473)
Epoch: [0][3/4]	Time 0.115 (0.496)	Data 0.001 (0.002)	Loss 1.4124 (2.1808)	MAE 0.413 (0.470)
Test: [0/2]	Time 0.048 (0.048)	Loss 77.1034 (77.1034)	MAE 3.420 (3.420)
 *	 Loss 77.7557 	 MAE 3.439
Epoch: [1][0/4]	Time 0.359 (0.359)	Data 0.002 (0.002)	Loss 0.9852 (0.9852)	MAE 0.284 (0.284)
Epoch: [1][1/4]	Time 0.374 (0.367)	Data 0.003 (0.003)	Loss 2.7863 (1.8857)	MAE 0.602 (0.443)
Epoch: [1][2/4]	Time 0.344 (0.359)	Data 0.003 (0.003)	Loss 1.2784 (1.6833)	MAE 0.317 (0.401)
Epoch: [1][3/4]	Time 0.082 (0.290)	Data 0.001 (0.002)	Loss 0.5262 (1.6206)	MAE 0.234 (0.392)
Test: [0/2]	Time 0.047 (0.047)	Loss 1.3342 (1.3342)	MAE 0.275 (0.275)
 *	 Loss 1.3533 	 MAE 0.283
Epoch: [2][0/4]	Time 0.356 (0.356)	Data 0.002 (0.002)	Los

In [9]:
#plt.plot(np.arange(len(loss_save)), loss_save)
#plt.show()